<h1 align="center">Segmenting & Clustering Neighbourhoods in Toronto</h1>

<h3>Part I - Web Scraping Wikipedia to a Dataframe</h3>

Part I corresponds to the first link submission (worth **10 marks**). Part II & III are located within this notebook and refer to the subsequent sections of the assignment.

To begin we need to import the packages we need for the dataframe (`pandas`) and web scraping (`requests`, `BeautifulSoup` and `lxml` - html parser).

In [27]:
import pandas as pd
import requests

!pip install beautifulsoup4
from bs4 import BeautifulSoup
!pip install lxml
from lxml import html

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Setting up the structure for the base dataframe.

In [17]:
column_names = ['PostalCode', 'Borough', 'Neighborhood']
neighborhoods = pd.DataFrame(columns = column_names)
neighborhoods

,PostalCode,Borough,Neighborhood


Scraping the Wikipedia information iterates over the table's rows and starts by testing both of the 'Not assigned' conditions. The second conditional checks if the postal code has been encountered before (based on the list of postal codes kept through the iterations of the rows in `postal_codes`) and concatenates the neighborhoods for the postal code into the existing dataframe where necessary.

In [18]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'html.parser')
table = soup.find('table')
table_body = table.find('tbody')
rows = table_body.find_all('tr')

postal_codes = []

for row in rows[1:]:
    cols = row.find_all('td')
    # cols[0] -> PostalCode, cols[1] -> Borough, cols[2] -> Neighborhood
    if (cols[1].text.strip() == 'Not assigned'):
        continue
    elif (cols[2].text.strip() == 'Not assigned'):
        cols[2] = cols[1]
        
    if (cols[0].text.strip() in postal_codes):
        i = postal_codes.index(cols[0].text.strip())
        neighborhoods.iloc[i]['Neighborhood'] = neighborhoods.iloc[i]['Neighborhood'] + ', ' + cols[2].text.strip()
    else:
        postal_codes.append(cols[0].text.strip())
        data = {
            'PostalCode': cols[0].text.strip(),
            'Borough': cols[1].text.strip(),
            'Neighborhood': cols[2].text.strip()
        }
        neighborhoods = neighborhoods.append(data, ignore_index = True)
       
neighborhoods

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


Now to show the number of rows in the dataframe:

In [19]:
neighborhoods.shape[0]

103

<h3>Part II - Retrieving Postal Code Latitude, Longitude</h3>

Part II corresponds to the second link submission (worth **2 marks**).

The `Geocoder` package was unreliable (long retrieval times, inconsistency), therefore the latitude, longitude data from the provided csv file was used.

In [5]:
!wget -q -O 'geospatial_coordinates.csv' http://cocl.us/Geospatial_data
print("Download complete")

geo_df = pd.read_csv("geospatial_coordinates.csv")
geo_df.head()

Download complete


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Based on the structure of the csv file data you can merge the `geo_df` dataframe to the existing `neighborhoods` dataframe on the postal codes. The redundant `'Postal Code'` column is then dropped after the merge.

In [6]:
neighborhoods_geo = pd.merge(neighborhoods, geo_df, left_on = 'PostalCode', right_on = 'Postal Code')
neighborhoods_geo.drop(columns = ['Postal Code'], inplace = True)
neighborhoods_geo

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509


<h3>Part III - Neighbourhood Clustering Analysis</h3>

Part III corresponds to the third and final link submission (worth **3 marks**)

In [7]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    sci

In [8]:
CLIENT_ID = 'RQMFZC0BRLLSPC2VXSDH2E45WUYJLE5FMWKUYPTHFIR0FRV3'
CLIENT_SECRET = 'PKCLP2UXF4IHAAOUKMUV02IUJADO3W55JT3JXVGC1ZORDFM5'
VERSION = '20180605'
LIMIT = 100
RADIUS = 500

In [9]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


The following is a `folium` map of Toronto (a base map to show the neighbourhood nodes to be used during analysis).

In [10]:
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 11)

for lat, lng, borough, neighborhood in zip(neighborhoods_geo['Latitude'], neighborhoods_geo['Longitude'], neighborhoods_geo['Borough'], neighborhoods_geo['Neighborhood']):
    label = '{} - {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)  
    
map_toronto

The method below is similar to that of the one used in the labs to return the nearby venues based on the specific neighbourhood. It makes use of the **Foursquare API** and corresponding parameters set above.

In [13]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            RADIUS, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

Using the method you're able to get nearby venues based on the parameters specified above.

In [14]:
toronto_venues = getNearbyVenues(neighborhoods_geo['Neighborhood'], neighborhoods_geo['Latitude'], neighborhoods_geo['Longitude'])
print(toronto_venues.shape)
toronto_venues.head(10)

(2229, 3)


,Neighborhood,Venue,Venue Category
0,Parkwoods,Brookbanks Park,Park
1,Parkwoods,GTA Restoration,Fireworks Store
2,Parkwoods,Variety Store,Food & Drink Shop
3,Victoria Village,Victoria Village Arena,Hockey Arena
4,Victoria Village,Tim Hortons,Coffee Shop
5,Victoria Village,Portugril,Portuguese Restaurant
6,Victoria Village,The Frig,French Restaurant
7,Victoria Village,Eglinton Ave E & Sloane Ave/Bermondsey Rd,Intersection
8,Victoria Village,Pizza Nova,Pizza Place
9,Harbourfront,Roselle Desserts,Bakery


In [33]:
toronto_categories = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_categories.drop(columns = ['Neighborhood'], inplace = True)
toronto_categories.insert(0, 'Neighborhood', toronto_venues['Neighborhood'])

toronto_categories.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [34]:
toronto_categories.shape

(2229, 273)

The following `groupby` sets up a dataframe for **K-Means** clustering.

In [36]:
toronto_avg = toronto_categories.groupby('Neighborhood').mean().reset_index()
toronto_avg.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.01,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.

In [37]:
toronto_avg.shape

(100, 273)

For the K-Means clustering, the number of clusters was set to **5** to allow for a moderate distribution of classifications while still exhibiting the underlying trend in the venue frequencies in the different neighbourhoods.

In [107]:
from sklearn.cluster import KMeans

k = 5

toronto_clustering = toronto_avg.drop('Neighborhood', 1)
kmeans = KMeans(init = 'k-means++', n_clusters = k, random_state = 1).fit(toronto_clustering)

kmeans.labels_[0:10]

array([2, 2, 1, 2, 0, 2, 2, 2, 2, 2], dtype=int32)

The final clustering classifications are then merged into the main dataframe (to form the `neighborhoods_kmeans` dataframe) for use by folium to generate a map displaying the clusters of the different neighbourhoods. 

In [108]:
df_kmeans = toronto_avg[['Neighborhood']]
df_kmeans.insert(1, 'Cluster Labels', kmeans.labels_)

neighborhoods_kmeans = pd.merge(neighborhoods_geo, df_kmeans, left_on = 'Neighborhood', right_on = 'Neighborhood')
neighborhoods_kmeans.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
0,M3A,North York,Parkwoods,43.753259,-79.329656,1
1,M4A,North York,Victoria Village,43.725882,-79.315572,0
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,2
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,2
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,2


The following processing is similar to that of the mapping provided in the labs and shows a predominant similarity throughout most of Toronto in regards to venue commonality.

In [109]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

map_toronto_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(k)
ys = [i + x + (i * x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, nbh, cluster in zip(neighborhoods_kmeans['Latitude'], neighborhoods_kmeans['Longitude'], neighborhoods_kmeans['Neighborhood'], neighborhoods_kmeans['Cluster Labels']):
    label = folium.Popup(str(nbh) + ' - Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_toronto_clusters)
       
map_toronto_clusters

The generated map shows that *the majority of downtown Toronto is structured fairly similarly* with a higher concentration of similarity on the coast in the heart of downtown Toronto. The following establishes the actual ordering of common venues per neighbourhood.

In [110]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [111]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind + 1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind + 1))

neighborhoods_venues_sorted = pd.DataFrame(columns = columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_avg['Neighborhood']

for ind in np.arange(toronto_avg.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_avg.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Thai Restaurant
1,Agincourt,Latin American Restaurant,Lounge,Breakfast Spot,Skating Rink,Yoga Studio
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Dumpling Restaurant,Diner,Discount Store
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fried Chicken Joint,Beer Store,Japanese Restaurant,Sandwich Place
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Skating Rink,Dance Studio,Pharmacy


The following dataframe(s) show the different clusters as to highlight any immediately apparent commonalities between the neighbourhoods. Based on an initial viewing it does not appear that there are any inherent differences amongst the majority of neighbourhoods.
<h4>Cluster 1</h4>

In [112]:
neighborhoods_venues_sorted.loc[neighborhoods_kmeans['Cluster Labels'] == 0, neighborhoods_venues_sorted.columns[[0] + list(range(1, neighborhoods_venues_sorted.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Agincourt,Latin American Restaurant,Lounge,Breakfast Spot,Skating Rink,Yoga Studio
10,"Bloordale Gardens, Eringate, Markland Wood, Ol...",Liquor Store,Pizza Place,Café,Coffee Shop,Convenience Store
18,Cedarbrae,Hakka Restaurant,Gas Station,Bank,Fried Chicken Joint,Caribbean Restaurant
50,Harbourfront,Coffee Shop,Bakery,Park,Pub,Restaurant
69,Northwest,Rental Car Location,Bar,Drugstore,Yoga Studio,Dim Sum Restaurant
71,"Parkdale, Roncesvalles",Gift Shop,Bookstore,Dog Run,Restaurant,Dessert Shop
76,"Rouge, Malvern",Fast Food Restaurant,Print Shop,Dumpling Restaurant,Discount Store,Dog Run
92,Westmount,Pizza Place,Middle Eastern Restaurant,Discount Store,Intersection,Sandwich Place


<h4>Cluster 2</h4>

In [113]:
neighborhoods_venues_sorted.loc[neighborhoods_kmeans['Cluster Labels'] == 1, neighborhoods_venues_sorted.columns[[0] + list(range(1, neighborhoods_venues_sorted.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Thai Restaurant
35,"Dovercourt Village, Dufferin",Bakery,Pharmacy,Park,Gym / Fitness Center,Middle Eastern Restaurant
45,"Flemingdon Park, Don Mills South",Gym,Coffee Shop,Beer Store,Sporting Goods Shop,Asian Restaurant
49,"Harbord, University of Toronto",Café,Restaurant,Bar,Italian Restaurant,Japanese Restaurant
60,"Kingsway Park South West, Mimico NW, The Queen...",Gym,Bakery,Grocery Store,Convenience Store,Discount Store
65,"Little Portugal, Trinity",Bar,Restaurant,Asian Restaurant,Coffee Shop,Pizza Place
84,"The Annex, North Midtown, Yorkville",Café,Sandwich Place,Coffee Shop,Park,Pharmacy
90,Thorncliffe Park,Indian Restaurant,Burger Joint,Yoga Studio,Bank,Grocery Store
96,Woburn,Coffee Shop,Soccer Field,Korean Restaurant,Yoga Studio,Eastern European Restaurant


<h4>Cluster 3</h4>

In [114]:
neighborhoods_venues_sorted.loc[neighborhoods_kmeans['Cluster Labels'] == 2, neighborhoods_venues_sorted.columns[[0] + list(range(1, neighborhoods_venues_sorted.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Dumpling Restaurant,Diner,Discount Store
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fried Chicken Joint,Beer Store,Japanese Restaurant,Sandwich Place
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Skating Rink,Dance Studio,Pharmacy
5,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Middle Eastern Restaurant,Gas Station,Fried Chicken Joint,Frozen Yogurt Shop
6,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Dog Run
7,"Bedford Park, Lawrence Manor East",Sandwich Place,Fast Food Restaurant,Italian Restaurant,Coffee Shop,Pizza Place
8,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Café,Farmers Market
9,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Diner
11,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Breakfast Spot,Grocery Store,Bakery
12,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Garden,Comic Shop,Park,Pizza Place


<h4>Cluster 4</h4>

In [115]:
neighborhoods_venues_sorted.loc[neighborhoods_kmeans['Cluster Labels'] == 3, neighborhoods_venues_sorted.columns[[0] + list(range(1, neighborhoods_venues_sorted.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
32,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Hotel,Café,Steakhouse,Bar
82,Stn A PO Boxes 25 The Esplanade,Coffee Shop,Café,Japanese Restaurant,Beer Bar,Hotel


<h4>Cluster 5</h4>

In [116]:
neighborhoods_venues_sorted.loc[neighborhoods_kmeans['Cluster Labels'] == 4, neighborhoods_venues_sorted.columns[[0] + list(range(1, neighborhoods_venues_sorted.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
56,"Humber Bay, King's Mill Park, Kingsway Park So...",Baseball Field,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant
99,York Mills West,Park,Bank,Convenience Store,Eastern European Restaurant,Dog Run
